In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist
from scipy.stats import rankdata
from collections import Counter
from copy import copy

In [3]:
#Read in excel file
df = pd.read_excel('/content/drive/MyDrive/23MentorMenteeSheets.xlsx')
df.columns = ["time", "email", "name", "mentor_mentee", "project", "data", "UIUX", "PM", "entrepreneurship", "computer_graphics", "leetcode", "fun"]

In [4]:
#Store the Name & Email
name_email_map = pd.Series(df.email.values, index=df.name).to_dict()

In [5]:
#Seperate the mentor and mentee
df_mentor = df[df['mentor_mentee'] == 'Mentor']
df_mentee = df[df['mentor_mentee'] == 'Mentee']

In [6]:
def findPairs(df_mentor, df_mentee):
  placeholder_list = []
  count = 0
  mentor_columns = df_mentor.columns.tolist()
  for x in range(len(df_mentee) - len(df_mentor)):
    placeholder_list.append(["N/A", "N/A" ,"Placeholder " + str(count), "Mentor", 0, 0, 0, 0, 0, 0, 0, 0])
    count += 1
  df_placeholder = pd.DataFrame(placeholder_list, columns = mentor_columns)
  df_mentor = pd.concat([df_mentor, df_placeholder])

  mentor_names = df_mentor['name']
  mentor_list = mentor_names.tolist()
  np_mentor = df_mentor.drop(['time','email','name', 'mentor_mentee'], axis = 1).to_numpy()

  mentee_names = df_mentee['name']
  mentee_list = mentee_names.tolist()
  np_mentee = df_mentee.drop(['time','email','name', 'mentor_mentee'], axis = 1).to_numpy()

  #Calculate the preference matrix
  preference_matrix= cdist(np_mentee,np_mentor, 'euclidean')

  #Find the resulting matrix
  final_preference = np.zeros([len(preference_matrix),len(preference_matrix[0])])

  for x in range(len(preference_matrix)):
    final_preference[x] = rankdata(preference_matrix[x], 'ordinal')


  mentee_df = pd.DataFrame(final_preference, index = mentee_list, columns = mentor_list)
  mentor_df = mentee_df

  #Gale Shapely matching ALGORITHM
  # dict to control which women each man can make proposals
  mentor_available = {mentee:mentor_list for mentee in mentee_list}
  # waiting list of men that were able to create pair on each iteration
  waiting_list = []
  # dict to store created pairs
  proposals = {}
  # variable to count number of iterations
  count = 0

  # while not all men have pairs
  while len(waiting_list)<len(mentee_list):
      # man makes proposals
      for mentee in mentee_list:
          if mentee not in waiting_list:
              # each man make proposal to the top women from it's list
              mentor = mentor_available[mentee]
              best_choice = mentee_df.loc[mentee][mentee_df.loc[mentee].index.isin(mentor)].idxmin()
              proposals[(mentee, best_choice)]=(mentee_df.loc[mentee][best_choice],
                                                  mentor_df.loc[mentee][best_choice])
      # if women have more than one proposals
      # she will choose the best option
      overlays = Counter([key[1] for key in proposals.keys()])
      # cycle to choose the best options
      for mentor in overlays.keys():
          if overlays[mentor]>1:
              # pairs to drop from proposals
              pairs_to_drop = sorted({pair: proposals[pair] for pair in proposals.keys()
                      if mentor in pair}.items(),
                    key=lambda x: x[1][1]
                    )[1:]
              # if man was rejected by woman
              # there is no pint for him to make proposal
              # second time to the same woman
              for p_to_drop in pairs_to_drop:
                  del proposals[p_to_drop[0]]
                  _mentor = copy(mentor_available[p_to_drop[0][0]])
                  _mentor.remove(p_to_drop[0][1])
                  mentor_available[p_to_drop[0][0]] = _mentor
      # man who successfully created pairs must be added to the waiting list
      waiting_list = [mentee[0] for mentee in proposals.keys()]
      # update counter
      count+=1

  return proposals



In [7]:
def findPairsMentees(df_mentor, df_mentee):
  placeholder_list = []
  count = 0
  mentee_columns = df_mentee.columns.tolist()
  for x in range(len(df_mentor) - len(df_mentee)):
    placeholder_list.append(["N/A", "N/A" ,"Placeholder " + str(count), "Mentee", 0, 0, 0, 0, 0, 0, 0, 0])
    count += 1
  df_placeholder = pd.DataFrame(placeholder_list, columns = mentee_columns)
  df_mentee = pd.concat([df_mentee, df_placeholder])

  mentor_names = df_mentor['name']
  mentor_list = mentor_names.tolist()
  np_mentor = df_mentor.drop(['time','email','name', 'mentor_mentee'], axis = 1).to_numpy()

  mentee_names = df_mentee['name']
  mentee_list = mentee_names.tolist()
  np_mentee = df_mentee.drop(['time','email','name', 'mentor_mentee'], axis = 1).to_numpy()
  #Calculate the preference matrix
  preference_matrix= cdist(np_mentee , np_mentor, 'euclidean')

  #Find the resulting matrix
  final_preference = np.zeros([len(preference_matrix),len(preference_matrix[0])])

  for x in range(len(preference_matrix)):
    final_preference[x] = rankdata(preference_matrix[x], 'ordinal')


  mentee_df = pd.DataFrame(final_preference, index = mentee_list, columns = mentor_list)
  mentor_df = mentee_df
  #Gale Shapely matching ALGORITHM
  # dict to control which women each man can make proposals
  mentor_available = {mentee:mentor_list for mentee in mentee_list}
  # waiting list of men that were able to create pair on each iteration
  waiting_list = []
  # dict to store created pairs
  proposals = {}
  # variable to count number of iterations
  count = 0

  # while not all men have pairs
  while len(waiting_list)<len(mentee_list):
      # man makes proposals
      for mentee in mentee_list:
          if mentee not in waiting_list:
              # each man make proposal to the top women from it's list
              mentor = mentor_available[mentee]
              best_choice = mentee_df.loc[mentee][mentee_df.loc[mentee].index.isin(mentor)].idxmin()
              proposals[(mentee, best_choice)]=(mentee_df.loc[mentee][best_choice],
                                                  mentor_df.loc[mentee][best_choice])
      # if women have more than one proposals
      # she will choose the best option
      overlays = Counter([key[1] for key in proposals.keys()])
      # cycle to choose the best options
      for mentor in overlays.keys():
          if overlays[mentor]>1:
              # pairs to drop from proposals
              pairs_to_drop = sorted({pair: proposals[pair] for pair in proposals.keys()
                      if mentor in pair}.items(),
                    key=lambda x: x[1][1]
                    )[1:]
              # if man was rejected by woman
              # there is no pint for him to make proposal
              # second time to the same woman
              for p_to_drop in pairs_to_drop:
                  del proposals[p_to_drop[0]]
                  _mentor = copy(mentor_available[p_to_drop[0][0]])
                  _mentor.remove(p_to_drop[0][1])
                  mentor_available[p_to_drop[0][0]] = _mentor
      # man who successfully created pairs must be added to the waiting list
      waiting_list = [mentee[0] for mentee in proposals.keys()]
      # update counter
      count+=1

  return proposals

In [ ]:
#df_mentor = df_mentor.sample(frac=1, random_state = 42)
#df_mentee = df_mentee.sample(frac=1, random_state = 42)

In [8]:
result = {}
count = 0
while len(df_mentee) >= len(df_mentor):
    iteration_result = findPairs(df_mentor, df_mentee)
    mentee_list = []
    for keys in iteration_result:
        mentor_name = keys[1]
        mentee_name = keys[0]
        # Lookup emails
        mentor_email = name_email_map.get(mentor_name, "Email not found")
        mentee_email = name_email_map.get(mentee_name, "Email not found")

        mentor_info = f"{mentor_name} ({mentor_email})"
        mentee_info = f"{mentee_name} ({mentee_email})"

        if 'Placeholder' not in mentor_name:
            if mentor_info in result:
                result[mentor_info].append(mentee_info)
            else:
                result[mentor_info] = [mentee_info]
        else:
            mentee_list.append(mentee_name)  # Keep using name for filtering in df_mentee
    df_mentee = df_mentee[df_mentee['name'].isin(mentee_list)]
    count += 1

if len(df_mentee) > 0:
    mentee_result = findPairsMentees(df_mentor, df_mentee)

    for keys in mentee_result:
        mentor_name = keys[1]
        mentee_name = keys[0]
        # Lookup emails
        mentor_email = name_email_map.get(mentor_name, "Email not found")
        mentee_email = name_email_map.get(mentee_name, "Email not found")

        mentor_info = f"{mentor_name} ({mentor_email})"
        mentee_info = f"{mentee_name} ({mentee_email})"

        if 'Placeholder' not in mentor_name:
            if mentor_info in result:
                result[mentor_info].append(mentee_info)
            else:
                result[mentor_info] = [mentee_info]

In [ ]:
result

In [9]:
final = pd.DataFrame(result.items(), columns=['Mentors', 'Mentees'])
final = final.explode('Mentees').reset_index(drop=True)

last_mentor = None
for index, row in final.iterrows():
    current_mentor = row['Mentors']
    if current_mentor == last_mentor:
        final.at[index, 'Mentors'] = ''  # Clear the mentor name
    else:
        last_mentor = current_mentor

final[['Mentor Name', 'Mentor Email']] = final['Mentors'].str.extract(r'(.*)\s\((.*)\)', expand=True)
final[['Mentee Name', 'Mentee Email']] = final['Mentees'].str.extract(r'(.*)\s\((.*)\)', expand=True)
final = final.drop(['Mentors', 'Mentees'], axis=1)
final.fillna('', inplace=True)
#final

In [ ]:
final_exploded.to_excel("/content/drive/MyDrive/MentorMenteeResult.xlsx")